In [26]:
from requests_html import HTML
from slugify import slugify
import pprint
import re
from app import scraper

In [27]:
url = "https://www.amazon.com/Acer-A515-56-32DK-Display-i3-1115G4-Processor/dp/B09R5VYRVN/ref=lp_16225007011_1_6?th=1"


In [28]:
s = scraper.Scraper(url = url, endless_scroll=True)

In [29]:
html_str = s.get()

In [30]:
html_obj = HTML(html = html_str)

In [31]:
def extract_element_text(html_obj, element_id):
    el =  html_obj.find(element_id, first=True)
    if not el:
        return ''
    return el.text

In [32]:
price_str = extract_element_text(html_obj, '.a-offscreen')
price_str


'$269.99'

In [33]:
title_str = extract_element_text(html_obj, "#productTitle")
title_str

'Acer Aspire 5 A515-56-32DK Slim Laptop - 15.6" Full HD IPS Display - 11th Gen Intel i3-1115G4 Dual Core Processor - 4GB DDR4 - 128GB NVMe SSD - WiFi 6\xa0- Amazon Alexa - Windows 11 Home in S mode.'

In [17]:
def extract_tables(html_obj):
    return html_obj.find("table")

In [22]:
tables = extract_tables(html_obj)
tables

[<Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-none', 'reInventPriceTable')>,
 <Element 'table' class=('a-lineitem',)>,
 <Element 'table' class=('a-normal', 'a-spacing-micro')>,
 <Element 'table' id='HLCXComparisonTable' class=('a-bordered', 'a-horizontal-stripes', 'a-spacing-none', 'a-size-base', 'comparison_table')>,
 <Element 'table' id='productDetails_detailBullets_sections1' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='productDetails_feedback_sections' class=('a-keyvalue', 'prodDetTable') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-spacing-none') role='presentation'>,
 <Element 'table' id='histogramTable' class=('a-normal', 'a-align-center', 'a-spacing-base')>,
 <Element 'table' class=('navFooterMoreOnAmazon',) cellspacing='0'>]

In [23]:
def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val

In [24]:
def extract_dataset(tables):
    dataset = {}
    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)
                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]
                
                # print(key, value)
                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if "$" in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = extract_price_from_string(value)
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    return dataset


In [25]:
extract_dataset(tables)

{'total': '$642.37',
 'total_raw': '$642.37',
 'brand': 'Vitamix',
 'color': 'Black',
 'special-feature': 'Not-Applicable',
 'capacity': '48 Fluid Ounces',
 'product-dimensions': '8"D x 11"W x 18"H',
 'material': 'Tritan',
 'included-components': "Container^ Lid^ Tamper^ Motor Base^ Owner's Manual^ Introduction Booklet",
 'style': 'Blender',
 'recommended-uses-for-product': 'Blending',
 'power-source': 'AC',
 'wattage': '1400 watts',
 'number-of-speeds': '10',
 'voltage': '120 Volts',
 'item-weight': '11.5 Pounds',
 'material-type-free': 'BPA Free',
 'is-dishwasher-safe': 'Yes',
 'blade-material': 'Stainless Steel',
 'manufacturer': 'Vitamix',
 'asin': 'B0758JHZM3',
 'country-of-origin': 'USA',
 'item-model-number': 'VM0197',
 'customer-reviews': '/* \n    * Fix for UDP-1061. Average customer reviews has a small extra line on hover \n    * https://omni-grok.amazon.com/xref/src/appgroup/websiteTemplates/retail/SoftlinesDetailPageAssets/udp-intl-lock/src/legacy.css?indexName=WebsiteTempl